In [1]:
import requests
from bs4 import BeautifulSoup
import time
from selenium.webdriver import Chrome
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

delay = 3
browser = Chrome("./chromedriver")
browser.implicitly_wait(delay)
browser.get("https://www.youtube.com/channel/UChE5nZAIhWS5vYTRjsUgRpQ/videos") # 박막례할머니 동영상 페이지로 들어감
browser.maximize_window()
body = browser.find_element_by_tag_name('body') # body태그 가져오기

num_of_pagedowns = 22
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1 #video페이지의 스크롤을 충분히 밑으로 내려놓고

In [2]:
html0 = browser.page_source
html = BeautifulSoup(html0, 'html.parser')
video_list2 = html.select('div #dismissable')

youtube_url = 'http://www.youtube.com'

park_url = []

#썸네일, 유형, 동영상 게시일, 영상길이, 제목, 조회수, 좋아요, 싫어요 를 저장하기 위한 리스트
thumbnail_list = []
category_list = []
upload_date_list = []
video_length_list = []
title_list = []
view_count_list = []
likes_count_list = []
dislikes_count_list = []


print("video_list2의 길이:",len(video_list2))
for i in range(len(video_list2)):
    url = youtube_url + video_list2[i].find('a',{'id': 'thumbnail'})['href']
    park_url.append(url) # 모든 영상의 url을 youtube_url에 붙여서 완전한 url로 만들고 park_url이라는 리스트에 넣어줌

video_list2의 길이: 478


In [3]:
for video in video_list2:
    video_length = video.select_one("ytd-thumbnail-overlay-time-status-renderer.ytd-thumbnail > span.style-scope")
    video_length = video_length.text.replace("\n","").strip()
    video_length_list.append(video_length)
    print(video_length)

11:23
7:22
11:06
6:44
5:54
6:57
19:36
10:07
11:14
6:47
12:36
10:04
6:59
8:02
10:41
5:24
8:25
6:31
15:17
3:56
10:47
13:19
8:32
17:31
12:59
10:38
6:04
14:08
12:36
6:25
10:06
11:41
5:53
10:50
8:46
1:02:16
11:08
12:17
7:48
10:35
7:03
8:37
10:08
10:02
14:33
7:56
12:10
6:37
10:04
10:14
6:31
8:10
13:37
11:58
10:44
5:58
10:45
5:57
3:25
11:22
6:57
10:42
10:53
10:11
10:19
10:06
10:52
5:36
12:00
10:25
10:35
5:24
10:03
10:07
10:40
10:30
11:19
10:32
12:51
8:22
10:20
11:39
4:36
11:03
10:25
10:24
2:33
6:44
10:06
8:46
7:33
11:13
10:06
10:16
3:34
7:11
7:49
6:47
10:03
10:42
6:06
10:06
10:17
10:05
6:34
4:30
10:13
10:08
3:47
4:48
10:04
10:44
10:48
4:13
3:39
6:17
10:08
10:03
10:05
4:47
6:15
8:12
7:53
6:33
10:05
10:09
6:59
8:44
10:03
10:07
5:16
13:50
10:12
10:11
10:07
10:12
5:56
10:12
10:28
4:50
12:04
10:05
11:09
4:23
7:27
3:15
10:03
10:05
16:25
6:50
6:06
6:58
10:05
10:04
6:48
8:15
7:59
8:14
11:23
10:10
10:05
4:30
10:07
10:05
6:24
10:11
10:04
10:04
7:19
10:04
4:48
10:03
10:16
3:07
10:08
1:29
7:05
6:29
20:07

In [4]:
# 현재로서는 썸네일, 유형, 동영상 게시일, 영상길이, 제목, 조회수, 좋아요, 싫어요, 만 크롤링 하면 될듯!

# park = pd.DataFrame({'thumbnail': [],
#                      'title' : [],
#                      'category': [],
#                      'view_count': [],
#                      'likes_count': [],
#                      'dislikes_count':[],
#                      'video_length':[],
#                      'upload_date': [],
#                      }) #총 8개의 정보

In [5]:
from selenium.common.exceptions import ElementNotInteractableException

for i in range(len(video_list2)):
    try:
        #여기까지는 video페이지에서 수집할 수 있는 썸네일, 제목을 가져온다.
        thumbnail = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src'] # 그 하위의 img라는 태그를 또 찾고 거기서 src가져와라 # 1
        title = video_list2[i].find('a',{'id':'video-title'}).text # 2 

        # 여기서부터 영상 클릭해서 들어간 페이지
        click_url = park_url[i]
        browser.get(click_url)
        time.sleep(1.5) # 페이지를 넘어가야하므로 로딩을 기다려야함
        # body = browser.find_element_by_tag_name('body')
        browser.find_element_by_xpath("//*[@id='more']/yt-formatted-string").click() # 카테고리를 크롤링 하기 위해서 더보기를 눌러주자
        time.sleep(3)
        html0 = browser.page_source
        html = BeautifulSoup(html0, 'html.parser')

        # 여기서는 카테고리, 조회수, 좋아요, 싫어요, 영상길이, 동영상 게시일을 크롤링한다 
        category = html.find('div', {'id':'collapsible'}).find('a',{'class':'yt-simple-endpoint'}).text
        # ytd-metadata-row-renderer.style-scope a

        view_count = html.find('span', {'class': 'view-count'}).text # 4

        # 좋아요와 싫어요는 완전히 똑같은 태그에 들어있다.
        # <파이썬 정규 표현식을 지원하는 >re모듈을 사용해보자
        likes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 5
                                                        'aria-label':re.compile('좋아요')}).text+'개'
        dislikes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 6
        'aria-label':re.compile('싫어요')}).text+'개'
        upload_date = html.find('div',{'id':'date'}).find('yt-formatted-string').text # 8

        print(i,".","썸네일", thumbnail, "제목:", title, "카테고리",category,"조회수",view_count,"좋아요수",likes_count, "싫어요수",dislikes_count, "게시일", upload_date)
    except ElementNotInteractableException:

        thumbnail = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src'] # 그 하위의 img라는 태그를 또 찾고 거기서 src가져와라 # 1
        title = video_list2[i].find('a',{'id':'video-title'}).text # 2 

        # 여기서부터 영상 클릭해서 들어간 페이지
        click_url = park_url[i]
        browser.get(click_url)
        time.sleep(1.5) # 페이지를 넘어가야하므로 로딩을 기다려야함
        # body = browser.find_element_by_tag_name('body')
        # browser.find_element_by_xpath("//*[@id='more']/yt-formatted-string").click() # 카테고리를 크롤링 하기 위해서 더보기를 눌러주자
        time.sleep(3)
        html0 = browser.page_source
        html = BeautifulSoup(html0, 'html.parser')

        # 여기서는 카테고리, 조회수, 좋아요, 싫어요, 영상길이, 동영상 게시일을 크롤링한다 
        # category = html.find('div', {'id':'collapsible'}).find('a',{'class':'yt-simple-endpoint'}).text
        # category = html.find('div', {'id':'always-shown'}).find('div',{'id':'content'}).find("a":{"class":"style-scope"}).text
        category = html.select_one("div#always-shown div#content a.style-scope")
        
        # ytd-metadata-row-renderer.style-scope a

        view_count = html.find('span', {'class': 'view-count'}).text # 4

        # 좋아요와 싫어요는 완전히 똑같은 태그에 들어있다.
        # <파이썬 정규 표현식을 지원하는 >re모듈을 사용해보자
        likes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 5
        'aria-label':re.compile('좋아요')}).text+'개'
        dislikes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 6
        'aria-label':re.compile('싫어요')}).text+'개'
        upload_date = html.find('div',{'id':'date'}).find('yt-formatted-string').text # 8

        print(i,".","썸네일", thumbnail, "제목:", title, "카테고리",category,"조회수",view_count,"좋아요수",likes_count, "싫어요수",dislikes_count, "게시일", upload_date)
    except AttributeError:
        #여기까지는 video페이지에서 수집할 수 있는 썸네일, 제목을 가져온다.
        thumbnail = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src'] # 그 하위의 img라는 태그를 또 찾고 거기서 src가져와라 # 1
        title = video_list2[i].find('a',{'id':'video-title'}).text # 2 

        # 여기서부터 영상 클릭해서 들어간 페이지
        click_url = park_url[i]
        browser.get(click_url)
        time.sleep(1.5) # 페이지를 넘어가야하므로 로딩을 기다려야함
        # body = browser.find_element_by_tag_name('body')
        browser.find_element_by_xpath("//*[@id='more']/yt-formatted-string").click() # 카테고리를 크롤링 하기 위해서 더보기를 눌러주자
        time.sleep(3)
        html0 = browser.page_source
        html = BeautifulSoup(html0, 'html.parser')

        # 여기서는 카테고리, 조회수, 좋아요, 싫어요, 영상길이, 동영상 게시일을 크롤링한다 
        category = html.find('div', {'id':'collapsible'}).find('a',{'class':'yt-simple-endpoint'}).text
        # ytd-metadata-row-renderer.style-scope a

        view_count = html.find('span', {'class': 'view-count'}).text # 4

        # 좋아요와 싫어요는 완전히 똑같은 태그에 들어있다.
        # <파이썬 정규 표현식을 지원하는 >re모듈을 사용해보자
        #likes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 5
        #                                                'aria-label':re.compile('좋아요')}).text+'개'
        #dislikes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 6
        #'aria-label':re.compile('싫어요')}).text+'개'
        likes_count = "0"
        dislikes_count = "0"
        upload_date = html.find('div',{'id':'date'}).find('yt-formatted-string').text # 8
        
        print(i,".","썸네일", thumbnail, "제목:", title, "카테고리",category,"조회수",view_count,"좋아요수",likes_count, "싫어요수",dislikes_count, "게시일", upload_date)
    
    
    finally:    
        thumbnail_list.append(thumbnail)
        title_list.append(title)
        category_list.append(category)
        view_count_list.append(view_count)
        likes_count_list.append(likes_count)
        dislikes_count_list.append(dislikes_count)
        upload_date_list.append(upload_date)

0 . 썸네일 https://i.ytimg.com/vi/QqyR7oBaXIE/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCSDvHOxKGvqLPf8TQTWnqD8nrr-g 제목: 하루종일 형 집 앞에서 몰래 배달음식 빼먹기 ㅋㅋㅋㅋ 배달원이 음식을 빼먹고 배달해주면 반응이 어떨까? 카테고리 코미디 조회수 조회수 601,001회 좋아요수 9.8천개 싫어요수 306개 게시일 2020. 1. 8.
1 . 썸네일 https://i.ytimg.com/vi/LhIcRFj44bw/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDjELFT9x1yvCboEho8WKNBZ7JJgw 제목: 24시간 굶고 쓰러지기 직전에 곤약젤리 1000개 먹방 ㅋㅋㅋㅋㅋㅋㅋ 드루와 카테고리 코미디 조회수 조회수 591,698회 좋아요수 1만개 싫어요수 339개 게시일 2020. 1. 6.
2 . 썸네일 https://i.ytimg.com/vi/d-9jvjcecK0/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBtMHPtIrHQeU5RF_qNDITXAK7ZgQ 제목: 30만원짜리 세상에서 제일 특이한 컴퓨터 직접 써봄 ㅋㅋㅋㅋㅋ 카테고리 코미디 조회수 조회수 962,096회 좋아요수 1.2만개 싫어요수 643개 게시일 2020. 1. 3.
3 . 썸네일 https://i.ytimg.com/vi/TJeJcjVc80U/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCZdsqz3MAgWIQgZKYPhtovob6Tvg 제목: 여자친구 앞에서 설거지하다가 갑자기 수세미 먹방 ㅋㅋㅋㅋㅋㅋ 과연 하또속? 카테고리 코미디 조회수 조회수 664,930회 좋아요수 1.2만개 싫어요수 3

33 . 썸네일 https://i.ytimg.com/vi/8HwcNLzqpEE/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLB23fcMtt-4r5pVKCo4UmfdT-IF1Q 제목: 24시간 굶고 쓰러지기 직전에 치즈폭탄밥 만들어먹기 ㅋㅋㅋㅋ [ 맛도리 가디언즈 ] 카테고리 코미디 조회수 조회수 758,976회 좋아요수 8.5천개 싫어요수 483개 게시일 2019. 11. 14.
34 . 썸네일 https://i.ytimg.com/vi/2L-UbpJsOt8/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCrIoGd1fqpP6TBj76_Q30czLG4rA 제목: 핫스팟 하나에 스마트폰 몇대까지 연결될까? 100대까지 연결해봄 ㅋㅋㅋㅋ [ 하찮은 실험실 feat.LG유플러스] 카테고리 코미디 조회수 조회수 313,484회 좋아요수 5.9천개 싫어요수 192개 게시일 2019. 11. 12.
35 . 썸네일 https://i.ytimg.com/vi/7H11yKmQGs4/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLA_mBROVOWjxOayxn1ch8T7s0I5lA 제목: 던질까말까 춤 1시간동안 쉬지않고 추기 ㅋㅋㅋㅋ feat.짱재영 카테고리 코미디 조회수 조회수 1,404,233회 좋아요수 2.5만개 싫어요수 1.1천개 게시일 최초 공개: 2019. 11. 9.
36 . 썸네일 https://i.ytimg.com/vi/nBDTtKgteyA/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAM37KeSDNSbxrp0_h7-HdKBZBYbQ 제목: 하루종일 다이어트 vs 살찌우기 2탄 ㅋㅋㅋㅋㅋ 과연 승자는 누구? 카테고리 코미디 조회수 조회

66 . 썸네일 https://i.ytimg.com/vi/K-qrqlO-SG0/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBFE_kKLqkJjT0eP4kUgd2VDV2AuQ 제목: 수박스테이크 vs 개꿀맛 스테이크 ㅋㅋㅋㅋ 복불복 먹방 [ 빈부격차 레스토랑 ] 카테고리 코미디 조회수 조회수 680,624회 좋아요수 7.3천개 싫어요수 420개 게시일 2019. 9. 25.
67 . 썸네일 https://i.ytimg.com/vi/X1SQNK1gycY/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBhzObDXRjAZUnVzPu1JfDWP3oOvg 제목: 라면 면발은 몇가닥일까? ㅋㅋㅋㅋ 알고 먹으면 더 맛있음 카테고리 코미디 조회수 조회수 420,929회 좋아요수 6.9천개 싫어요수 277개 게시일 2019. 9. 23.
68 . 썸네일 https://i.ytimg.com/vi/Ky72wAkujRA/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDsp4LKTko4Mo7GqLjgrfiASMGxWA 제목: 하루종일 영어만쓰기 vs 한국어만쓰기 ㅋㅋㅋㅋㅋㅋ 어떻게 될까? 카테고리 코미디 조회수 조회수 1,826,671회 좋아요수 1.7만개 싫어요수 931개 게시일 2019. 9. 21.
69 . 썸네일 https://i.ytimg.com/vi/0dOIe1az3sk/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCedSRykHYpSv3boQ0_DPCwzMvo8Q 제목: 솥뚜껑에 삼겹살 구워서 고문먹방 ㅋㅋㅋㅋㅋ 카테고리 코미디 조회수 조회수 721,694회 좋아요수 7.8천개 싫어요수 415개 게시일 2019. 9. 20.
70 . 썸네일 

99 . 썸네일 https://i.ytimg.com/vi/XKJ3GjWk0pA/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAOX1Jywy-Hz6MnZ3c-SNbayKAiUg 제목: 하루종일 연예인으로 살기 vs 매니저로 살기 ㅋㅋㅋㅋㅋㅋ [ 극한직업 공피디 ] 카테고리 코미디 조회수 조회수 1,138,748회 좋아요수 1.4만개 싫어요수 828개 게시일 최초 공개: 2019. 8. 12.
100 . 썸네일 https://i.ytimg.com/vi/UQ9FShAXyqI/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAKgzDMxmKJ7X7i1xciU8QPFMyDSg 제목: 샴푸 한번 짜는데 얼마? 카테고리 코미디 조회수 조회수 253,800회 좋아요수 5.4천개 싫어요수 174개 게시일 최초 공개: 2019. 8. 11.
101 . 썸네일 https://i.ytimg.com/vi/VJ78DR1MKn0/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBpRC4GSqpt94EDXQgTQ1nDX4iHYA 제목: 콜라로 할 수 있는 기발한 장난 10가지 ㅋㅋㅋㅋ 초간단! 카테고리 코미디 조회수 조회수 1,005,919회 좋아요수 1.3만개 싫어요수 698개 게시일 2019. 8. 10.
102 . 썸네일 https://i.ytimg.com/vi/QIwYxX42UTg/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDi7procPw-FHx5I0f8jzGqkIvLXQ 제목: 하루종일 '네발로 살기' vs '한발로 살기' ㅋㅋㅋㅋ 뭐가 더 불편할까? 카테고리 코미디 조회수 조회수 747,451회 좋아요수 1.1만개 싫어요수 572개 게시일 최초 공개:

132 . 썸네일 https://i.ytimg.com/vi/BF_p6b3lqRg/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCzA95t0lK-oCIimmJXR3dk_vj6IQ 제목: 택시미터기 100만원 넘어갈까? 직접 타봄 ㅋㅋㅋㅋㅋ 카테고리 코미디 조회수 조회수 1,911,884회 좋아요수 2.7만개 싫어요수 1.1천개 게시일 2019. 6. 25.
133 . 썸네일 https://i.ytimg.com/vi/UCQxRYgrwwk/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLC8SkbD0TE6rL020mb62mPbJMsdlA 제목: 100만원짜리 커플룩 vs 1만원짜리 커플룩 ㅋㅋㅋ 뭐가 더 이쁠까? [ 그들의 삶 6탄 ] 카테고리 코미디 조회수 조회수 388,696회 좋아요수 1만개 싫어요수 200개 게시일 2019. 6. 22.
134 . 썸네일 https://i.ytimg.com/vi/Ueu90SvlELI/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDTg-bK8V99EDoj44OkV1SUV4cmog 제목: ? 카테고리 코미디 조회수 조회수 173,734회 좋아요수 5천개 싫어요수 105개 게시일 2019. 6. 21.
135 . 썸네일 https://i.ytimg.com/vi/BqPAeNDTG88/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAP40op4VZD8opzq0WxBm0HU16MQA 제목: [칠대죄] 폰게임에 500만원 현질해서 랭킹 1위 먹었습니다 ㅋㅋㅋㅋㅋㅋ 카테고리 게임 조회수 조회수 306,042회 좋아요수 0 싫어요수 0 게시일 2019. 6. 20.
136 . 썸네일 https://i.ytimg.co

...

373 . 썸네일 https://i.ytimg.com/vi/DvbOkwsmddM/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBoShm_HE0eyoikfx96cIR_tIHI_w 제목: 불에 구워먹는 아이스크림 만들어먹기 ㅋㅋㅋㅋㅋㅋ 대신기 ㅋㅋㅋㅋ [ 신쿡의 요리교실 ?! ] 공대생 변승주 카테고리 엔터테인먼트 조회수 조회수 1,999,513회 좋아요수 2만개 싫어요수 624개 게시일 2018. 7. 11.
374 . 썸네일 https://i.ytimg.com/vi/AmCBCK_7YD4/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDeUThC3jOaSIqIQRE4wlx1UtI3wA 제목: 스마트폰에 컴퓨터용 키보드랑 마우스 연결해서 게임하면 신세계임 ㅋㅋㅋㅋ feat. 모바일 배틀그라운드 [ 스마트폰을 노트북으로 만드는 방법 ] 공대생 변승주 카테고리 엔터테인먼트 조회수 조회수 612,821회 좋아요수 8.8천개 싫어요수 296개 게시일 2018. 7. 10.
375 . 썸네일 https://i.ytimg.com/vi/k4On9VwQ6MM/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDwFG10b2Kx8I6Rn5Ufs6BTqcns-g 제목: 생전 처음 아이돌 메이크업 받고 여자친구한테 보여줬더니 반응이 ㅋㅋㅋㅋㅋ [ 공대생 아이돌 만들기 프로잭트 part2 ] 공대생 변승주 카테고리 엔터테인먼트 조회수 조회수 1,370,813회 좋아요수 1.9만개 싫어요수 793개 게시일 2018. 7. 9.
376 . 썸네일 https://i.ytimg.com/vi/InvGh4myRN8/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLA8B3RKt

402 . 썸네일 https://i.ytimg.com/vi/qJBLnjBh2mM/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDHojQiF6gltz4AKe7cxWHAjbNZmg 제목: 생전 처음으로 여자친구랑 커플마사지를 받아보았습니다.. ㅎㅎㅎㅎ [ 01커플 세부여행 3탄 ] 공대생 변승주 카테고리 엔터테인먼트 조회수 조회수 286,694회 좋아요수 6.4천개 싫어요수 88개 게시일 2018. 6. 10.
403 . 썸네일 https://i.ytimg.com/vi/jUXFOuHEC84/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCq4PVE2bFTVTXi8nsEGuSjuWUrOg 제목: 여자친구랑 바다 놀러갔다가 초거대 상어가 나타났습니다 ㅋㅋㅋㅋ 실화냐? ㅋㅋㅋㅋ  [ 01커플 세부여행 2편 ] 공대생 변승주 카테고리 엔터테인먼트 조회수 조회수 1,581,384회 좋아요수 1.5만개 싫어요수 532개 게시일 2018. 6. 9.
404 . 썸네일 https://i.ytimg.com/vi/HSKDX_JvqOg/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLD19lWg0MzdVnNH6LkPwAflFBQYcw 제목: 팬들이랑 숨바꼭질 ㅋㅋㅋㅋ 가장 먼저 찾는 사람 문화상품권 10만원 준다했는데 공항 마비됨 ㅋㅋㅋㅋ  [ 01커플 세부여행 1편 ] 공대생 변승주 카테고리 엔터테인먼트 조회수 조회수 242,351회 좋아요수 6.3천개 싫어요수 83개 게시일 2018. 6. 8.
405 . 썸네일 https://i.ytimg.com/vi/FrBH2cnZFnA/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLD6c3bKLhDmub-g02pWMb

431 . 썸네일 https://i.ytimg.com/vi/z7PLkmO437c/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAurwXQYMXzQVBSYFglBW_2nOJebQ 제목: 여자친구가 대구 꿀떡이 먹고싶다고 해서 대구까지 가서 사왔습니다 ㅋㅋㅋㅋ [ 대구 꿀떡 먹방 ] 공대생 변승주 카테고리 엔터테인먼트 조회수 조회수 1,367,053회 좋아요수 1.6만개 싫어요수 627개 게시일 2018. 5. 11.
432 . 썸네일 https://i.ytimg.com/vi/EbzJ-YvShos/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBSNME8n1wGvk3Y6iPPllgIi8Oyrw 제목: 3M짜리 초거대 음료수가 있다고 해서 직접 찾아가 봤습니다 ㅋㅋㅋ [ 초거대 음료수 안에 들어가보기. 방탈출을 넘어서 병탈출? 레몬1000c ] 공대생 변승주 카테고리 엔터테인먼트 조회수 조회수 912,603회 좋아요수 1.2만개 싫어요수 320개 게시일 2018. 5. 10.
433 . 썸네일 https://i.ytimg.com/vi/RxMD4QZwFt8/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLD7ZWTCOSXevDk1NBowHVKyK5QOgA 제목: 냉장고를 열었는데 갑자기 500만원이 있으면 엄마의 반응이 어떨까? ㅋㅋㅋㅋㅋㅋ [ 엄마 놀래키기 + 용돈드리기 2탄 ] 공대생 변승주 카테고리 엔터테인먼트 조회수 조회수 502,304회 좋아요수 1만개 싫어요수 120개 게시일 2018. 5. 9.
434 . 썸네일 https://i.ytimg.com/vi/0nXjmbgwV1c/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAjQgEw

460 . 썸네일 https://i.ytimg.com/vi/b6zc_js2Ar8/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCvYccf_n5ByZvrFnNl3Kvq30vf6w 제목: 미용실에서 다짜고짜 박보검처럼 만들어달라고 하면 반응이 어떨까? ㅋㅋㅋㅋ [ 밥버거 머리 하고왔어요^^ with 로미쌤 ] 공대생 변승주 카테고리 엔터테인먼트 조회수 조회수 355,513회 좋아요수 6.6천개 싫어요수 155개 게시일 2018. 4. 5.
461 . 썸네일 https://i.ytimg.com/vi/nVBPGDZjDiM/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAgdXfzvEsaQOKG20HFFoPpvtcSSg 제목: 테이프로 스마트폰 카메라 필터 직접 만들어서 유니크한 사진 찍어보기 ㅋㅋㅋㅋ 개신기함 [ 스마트폰을 혁신적으로 쓰는 방법.. ] 공대생 변승주 카테고리 엔터테인먼트 조회수 조회수 542,032회 좋아요수 8천개 싫어요수 227개 게시일 2018. 4. 3.


KeyError: 'src'

In [6]:
print(len(thumbnail_list),len(video_length_list),len(title_list))

463 478 463


In [ ]:
import copy
video_length_list_copy = copy.deepcopy(video_length_list[:len(thumbnail_list)])
len(video_length_list_copy), len(video_length_list)

In [ ]:
import pandas as pd

insert_data = pd.DataFrame({
                    'thumbnail': thumbnail_list,
                    'title' : title_list,
                    'category': category_list,
                    'view_count': view_count_list,
                    'likes_count': likes_count_list,
                    'dislikes_count':dislikes_count_list,
                    'upload_date': upload_date_list,
                    'video_length' : video_length_list_copy
                    }) 
insert_data

In [ ]:
import openpyxl
insert_data.to_excel("공대생변승주.xlsx",encoding="utf8")

In [ ]:
# 소련여자, 박막례, 최고다윽박, ondo온도, 오킹, 오늘의 하늘, 박병창
# 슛뚜 미트 난치 이승인 공대생변승주  정선호

In [ ]:
df